In [1]:
from langchain_openai import ChatOpenAI

# Initialize the model
llm = ChatOpenAI()
print("ChatOpenAI model initialized.")

ChatOpenAI model initialized.


In [2]:
# Invoke simple request about langsmith
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can generate test cases automatically, reducing the time and effort required for manual testing. It can create a large number of test cases by varying input parameters and generate test data based on predefined rules.\n\n2. Test Coverage: Langsmith can analyze the code and identify areas that are not covered by existing tests. It can suggest additional test cases to improve code coverage and identify potential gaps in the testing strategy.\n\n3. Regression Testing: Langsmith can automatically generate test cases to ensure that code changes or updates do not introduce new bugs or regressions. It can compare the output of the modified code with the expected output to detect any discrepancies.\n\n4. Performance Testing: Langsmith can simulate different scenarios and generate test cases to evaluate the performance of the software under various conditions. It can measure response times, id

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer. End the response with 'Good Day Mate!'"),
    ("user", "{input}")
])
# Combine these into a simple LLM chain
chain = prompt | llm 
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith is a powerful testing tool that can greatly assist in ensuring the quality and reliability of software applications. Here are some ways in which Langsmith can help with testing:\n\n1. Test Automation: Langsmith allows you to automate the execution of tests, saving time and effort compared to manual testing. It provides a user-friendly interface for creating, managing, and executing automated test scripts.\n\n2. Test Case Management: With Langsmith, you can easily create and organize test cases, test suites, and test plans. It provides a centralized repository for storing and managing all testing artifacts, making it easier to track progress and ensure test coverage.\n\n3. Test Data Management: Langsmith offers capabilities for managing test data, including creating and maintaining test data sets. This helps in ensuring that tests are executed with a variety of data scenarios, covering different use cases.\n\n4. Test Execution and Reporting: Langsmith allows

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "Your name is Jenny."),
    ("human", "Hello, how are you?"),
    ("ai", "I'm doing well, thanks!"),
    ("user", "{input}")
])
# Combine these into a simple LLM chain
chain2 = prompt2 | llm 
chain2.invoke({"input": "Hello, who are you?"})

AIMessage(content='Hello, I am an AI assistant. You can call me Jenny. How can I assist you today?')

In [5]:
# Adding str parser at the end of chain to parse output as string
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can greatly assist with testing by providing a range of useful features and tools. Firstly, it offers automated testing capabilities which can help streamline the testing process and improve efficiency. With Langsmith, you can easily write test scripts using its user-friendly scripting language, making it simple to create and execute tests.\n\nAdditionally, Langsmith provides comprehensive reporting functionality, allowing you to generate detailed test reports that highlight any issues or failures encountered during testing. These reports can be invaluable in identifying areas that require further attention or improvement.\n\nMoreover, Langsmith supports integration with popular testing frameworks and tools, enabling seamless integration into your existing testing environment. It also offers support for different platforms and devices, ensuring that your tests can be executed across various operating systems and hardware configurations.\n\nLastly, Langsmith offers excellent 

## Retrieval Chain

In [6]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

# Use embedding model from OpenAI
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# Build the index into vector store
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [7]:
# Setup the document chain with the prompt template
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

# Run a query with 1-liner page content
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

### Using the vector store created earlier

In [8]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing the ability to run a chain over data points and visualize the outputs. This allows for manual inspection of the results and evaluation of the model's performance. LangSmith also provides evaluators that can be used to automatically evaluate the results of test runs. Additionally, LangSmith makes it easy to assign feedback to runs and mark them as correct or incorrect, providing aggregate statistics for each test project.


## Conversation Retrieval Chain

In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [10]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='LangSmith Overview and User Guide | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content='Skip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used i

In [11]:
## Create a prompt template that take chat_history
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

## New retrieval chain that
## 1. Retrieve context / content
## 2. Load LLM (OpenAI Chat)
## 3. Prompt template with chat history
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [12]:
## Run a test of LangChain with chat history
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='LangSmith Overview and User Guide | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
  Document(page_content='Skip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the i

# Agent

In [13]:
## Create a retriever tool to answer questions about LangSmith.
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [14]:
## Create Tavily Search tool
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

In [15]:
tools = [retriever_tool, search_tool]

In [16]:
## Create an agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
## Test LangSmith related question
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the execution of te

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been tested and may require additional test cases.\n\n4. Test Prioritization: LangSmith can 

In [18]:
## Test non-LangSmith related question, it should use Tavily search
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/', 'content': 'San Francisco Weather in January  San Francisco weather in January San Francisco weather by month // weather averages 9.6 (49.2) 6.2 (43.2) 14 (57.3) 113  you can find all information about the weather in San Francisco in January:  San Francisco weather in January // weather averages Airport close to San FranciscoData: 1991 - 2021 Min. Temperature °C (°F), Max. Temperature °C (°F), Precipitation / Rainfall mm (in), Humidity, Rainy days. Data: 1999 - 2019: avg. Sun hours San Francisco weather and climate for further months San Francisco in February San Francisco in March San Francisco in April San Francisco in May San Francisco in June San Francisco in July'}]The weather in San Francisco can vary depending on the time of year. In J

{'input': 'what is the weather in SF?',
 'output': 'The weather in San Francisco can vary depending on the time of year. In January, the average minimum temperature is around 9.6°C (49.2°F) and the average maximum temperature is around 14°C (57.3°F). The city experiences an average of 113mm (4.4 inches) of precipitation during this month. If you would like more information about the weather in San Francisco for other months, please let me know.'}

In [19]:
## Can add chat history
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, grammar, and overall impact.

2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and adheres to the standard format for LLM applications.

3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders, as well as tips on how to request strong recommendation letters. Additionally, LangSmith can review draft recommendation letters to ensure they effectively highlight your strengths and qualifications.

4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting th

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, grammar, and overall impact.\n\n2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and adheres to the standard format for LLM applications.\n\n3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders, as well as tips on how to request strong recommendation letters. Additionally, LangSmith can review draft recommendation letters to ensure they effectively highlight your strengths and qualificatio

# Tools

In [20]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [21]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [25]:
wiki_tool.name

'Wikipedia'

In [27]:
wiki_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [29]:
wiki_tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [31]:
wiki_tool.return_direct

False

In [34]:
wiki_tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [35]:
wiki_tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

### Customizing Default Tools

In [36]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)
print(tool.name)
print(tool.description)
print(tool.args)
print(tool.return_direct)
print(tool.run("langchain"))

wiki-tool
look up things in wikipedia
{'query': {'title': 'Query', 'description': 'query to look up in Wikipedia, should be 3 or less words', 'type': 'string'}}
True
Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 
